# LSTM

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

cpu is available


In [ ]:
input_size = 5
num_layers = 2
hidden_size = 8
sequence_length=5

In [ ]:
import torch
from torch.autograd import Variable
from torch import nn

class lstm(nn.Module):
  def __init__(self,input_size, hidden_size):   # input 5,  hidden 8
    super(lstm,self).__init__()

    self.Wf=nn.Parameter(torch.randn(1, input_size))            
    self.bf=nn.Parameter(torch.randn(1, hidden_size))            # (8,1)
    self.Wi=nn.Parameter(torch.randn(1,input_size))             # (8,1)   
    self.bi=nn.Parameter(torch.randn(1,hidden_size))            # (1,8)
    self.Wc=nn.Parameter(torch.randn(1,input_size))             # (8,1)   
    self.bc=nn.Parameter(torch.randn(1,hidden_size)) 
    self.Why=nn.Parameter(torch.randn(1,hidden_size))           # (1,8) * (8,1) = (1,1)
    self.by=nn.Parameter(torch.randn(1,1))                       # (1,1)
    self.Wo=nn.Parameter(torch.randn(1,input_size))
    self.bo=nn.Parameter(torch.randn(1,hidden_size))

    self.tanh=nn.Tanh()
    self.sig=nn.Sigmoid()
  
  def forward(self,x,h,c):
    f=self.tanh(self.Wf.matmul((x.matmul(h))) + self.bf)    #  (1,5) (5,8)  (1,8)
    i=self.sig(self.Wi.matmul((x.matmul(h))) + self.bi)
    ct=self.tanh(self.Wc.matmul((x.matmul(h))) + self.bc) 
    o=self.sig(self.Wo.matmul((x.matmul(h)))+self.bo)

    c=c.mul(f)+i.mul(ct)
    h=self.tanh(c).mul(o)
    y=h*self.Why+self.by
    return y, (c,h)


class LSTM(nn.Module):

  def __init__(self, backbone, input_size, hidden_size, sequence_length, num_layers, device):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = lstm(input_size, hidden_size)
    self.fc = nn.Sequential(nn.Linear(hidden_size , 1), nn.Sigmoid())
    self.device=device

  def forward(self, x):
    h0 = torch.zeros(x.size()[1], self.hidden_size).to(self.device) # 초기 hidden state 설정하기.
    c0 = torch.zeros(x.size()[1], self.hidden_size).to(self.device) # 이거 추가 -> rnn과 다른점
    out, (_,_) = self.lstm(x, h0,c0) # out: RNN의 마지막 레이어로부터 나온 output feature 를 반환한다. hn: hidden state를 반환한다.
    out = out.reshape(out.shape[0], -1) # many to many 전략
    out = self.fc(out)
    return out

model = LSTM(lstm, input_size,hidden_size,sequence_length,num_layers, device)
                  

In [ ]:
model

LSTM(
  (lstm): lstm(
    (tanh): Tanh()
    (sig): Sigmoid()
  )
  (fc): Sequential(
    (0): Linear(in_features=8, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [ ]:
x=torch.randn(5,1)
input = x
out = model(x)
print('input : ', input.size())
print('output : ', out.size())

input :  torch.Size([5, 1])
output :  torch.Size([1, 1])
